In [43]:
import pandas as pd


class Coordinate:
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat

    def get_lon(self):
        return self.lon
    
    def get_lat(self):
        return self.lat
    
    def get_lat_lon(self):
        return (self.lat, self.lon)

In [49]:
df_hdb = pd.read_csv('../data/modified/hdb_coordinates.csv')
df_hdb = df_hdb.drop(columns=['Unnamed: 0'], axis=1)
df_hdb

,address,latitude,longitude
0,174 ANG MO KIO AVE 4,1.375097,103.837619
1,541 ANG MO KIO AVE 10,1.373922,103.855621
2,163 ANG MO KIO AVE 4,1.373549,103.838176
3,446 ANG MO KIO AVE 10,1.367761,103.855357
4,557 ANG MO KIO AVE 10,1.371626,103.857736
...,...,...,...
7916,634 WOODLANDS RING RD,1.436266,103.802943
7917,810 WOODLANDS ST 81,1.442668,103.788139
7918,844 YISHUN ST 81,1.415625,103.835447
7919,654 YISHUN AVE 4,1.423053,103.840410


### Calculate distances for POI
Purpose: To calculate distances from addresses to POI
- distance2cbd
- distance2nearestschool
- distance2nearestmrt

How: Use OpenRouteService API to get distances

##### distance2cbd

In [50]:
from geopy.distance import geodesic

def calculate_distance(start: Coordinate, end: Coordinate) -> float:
    return geodesic(start.get_lat_lon(), end.get_lat_lon()).km

In [51]:
CENTRAL_BUSINESS_DISTRICT = Coordinate(lon=103.851784, lat=1.287953)
A_174AMKAVE4 = Coordinate(lat=1.37509746867904, lon=103.83761896123)

print(calculate_distance(A_174AMKAVE4, CENTRAL_BUSINESS_DISTRICT))

9.764086519543062


In [52]:
df_hdb['distance2cbd'] = df_hdb.apply(lambda row: calculate_distance(CENTRAL_BUSINESS_DISTRICT, Coordinate(lat=row['latitude'], lon=row['longitude'])), axis=1)
df_hdb.head(10)

,address,latitude,longitude,distance2cbd
0,174 ANG MO KIO AVE 4,1.375097,103.837619,9.764087
1,541 ANG MO KIO AVE 10,1.373922,103.855621,9.515642
2,163 ANG MO KIO AVE 4,1.373549,103.838176,9.585102
3,446 ANG MO KIO AVE 10,1.367761,103.855357,8.833708
4,557 ANG MO KIO AVE 10,1.371626,103.857736,9.275781
5,603 ANG MO KIO AVE 5,1.380201,103.835756,10.355083
6,709 ANG MO KIO AVE 8,1.371137,103.847662,9.209514
7,333 ANG MO KIO AVE 1,1.361343,103.851699,8.115046
8,109 ANG MO KIO AVE 4,1.370097,103.837688,9.217490
9,564 ANG MO KIO AVE 3,1.369848,103.859404,9.095193


In [ ]:
df_hdb.to_csv('../data/modified/hdb_distance2cbd.csv')

#### Archive Code

In [18]:
import os
import requests
from dotenv import load_dotenv


load_dotenv()
API_KEY = os.getenv("ONEMAP_SG_API")


def get_distance_between_two_coordinates(start: Coordinate, end: Coordinate) -> float:
    # &maxWalkDistance=1000&numItineraries=3

    date = "08-13-2023"

    url = \
f"""
https://www.onemap.gov.sg/api/public/routingsvc/route?start={start.get_lat()}%2C{start.get_lon()}&end={end.get_lat()}%2C{end.get_lon()}&routeType=pt&date={date}&time=07%3A35%3A00&mode=TRANSIT&maxWalkDistance=1000&numItineraries=3
"""
      
    headers = {"Authorization": API_KEY}
        
    response = requests.request("GET", url, headers=headers)
        
    print(response.text)



In [19]:
CENTRAL_BUSINESS_DISTRICT = Coordinate(lon=103.851784, lat=1.287953)
A_174AMKAVE4 = Coordinate(lat=1.37509746867904, lon=103.83761896123)

print(get_distance_between_two_coordinates(A_174AMKAVE4, CENTRAL_BUSINESS_DISTRICT))

{
  "status": "error",
  "message": "Token has expired: Session expired. Please refresh your token (if still within refresh window) or re-login."
}
None


In [10]:
import os
import requests

from dotenv import load_dotenv


load_dotenv()
API_KEY_2 = os.getenv("OPEN_ROUTE_SERVICE_API_KEY")


def get_distance_between_two_coordinates(start: Coordinate, end: Coordinate) -> float:
    # Create the URL for the Directions API
    try:
        url = 'https://api.openrouteservice.org/v2/directions/driving-car'

        params = {
            'api_key': API_KEY_2,
            'start': f"{start.get_lon()},{start.get_lat()}",
            'end': f"{end.get_lon()},{end.get_lat()}"
        }

        # Make the request
        response = requests.get(url, params=params)
        print(response)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the distance from the response (in meters)
            distance = data['features'][0]['properties']['segments'][0]['distance']

            # convert distance to kilometers
            distance = distance / 1000
        else:
            distance = -1
        return distance
    except Exception:
        return -2

In [28]:
CENTRAL_BUSINESS_DISTRICT = Coordinate(lon=103.851784, lat=1.287953)

address_list = df_hdb['address'].tolist()
latitude_list = df_hdb['latitude'].tolist()
longitude_list = df_hdb['longitude'].tolist()
distance_list = []

for i, address in enumerate(address_list):
    hdb_coordinate = Coordinate(lat=latitude_list[i], lon=longitude_list[0])
    distance = calculate_distance(hdb_coordinate, CENTRAL_BUSINESS_DISTRICT)

    distance_list.append(distance)

    print(f"Distance to CBD from {address} : {distance}")

distance_list

Distance to CBD from 174 ANG MO KIO AVE 4 : 9.764086519543062
Distance to CBD from 541 ANG MO KIO AVE 10 : 9.635879511274803
Distance to CBD from 163 ANG MO KIO AVE 4 : 9.595100676141982
Distance to CBD from 446 ANG MO KIO AVE 10 : 8.964451387368994
Distance to CBD from 557 ANG MO KIO AVE 10 : 9.385437424022244
Distance to CBD from 603 ANG MO KIO AVE 5 : 10.321384563763871
Distance to CBD from 709 ANG MO KIO AVE 8 : 9.332194032150701
Distance to CBD from 333 ANG MO KIO AVE 1 : 8.266740817411446
Distance to CBD from 109 ANG MO KIO AVE 4 : 9.21879625472943
Distance to CBD from 564 ANG MO KIO AVE 3 : 9.191761147054729
Distance to CBD from 218 ANG MO KIO AVE 1 : 8.677031231596597
Distance to CBD from 556 ANG MO KIO AVE 10 : 9.42976759057876
Distance to CBD from 156 ANG MO KIO AVE 4 : 9.807490738638814
Distance to CBD from 471 ANG MO KIO AVE 10 : 8.497350204427836
Distance to CBD from 434 ANG MO KIO AVE 10 : 8.978663393915737
Distance to CBD from 560 ANG MO KIO AVE 10 : 9.297218072079
Dista

[9.764086519543062,
 9.635879511274803,
 9.595100676141982,
 8.964451387368994,
 9.385437424022244,
 10.321384563763871,
 9.332194032150701,
 8.266740817411446,
 9.21879625472943,
 9.191761147054729,
 8.677031231596597,
 9.42976759057876,
 9.807490738638814,
 8.497350204427836,
 8.978663393915737,
 9.297218072079,
 8.3023196847869,
 8.705863953163304,
 9.688029545638368,
 10.251069291377679,
 9.90160724750195,
 8.735123215580934,
 9.342975656866312,
 9.077967929268425,
 9.937651680530047,
 8.350211393931259,
 9.175844752121524,
 9.251632119637746,
 10.233358135120671,
 9.056517396891747,
 9.285973499835109,
 8.53963338630603,
 10.180125589650922,
 9.228046430281982,
 8.778066548683604,
 9.950661097749455,
 8.650682976194526,
 9.84493490747064,
 8.604654804076707,
 9.230443132112704,
 8.753354697066992,
 9.024219749322107,
 9.613484572463497,
 9.09272033046141,
 9.269449765406108,
 8.658214114284238,
 9.02682754854928,
 9.431825255795435,
 8.901534601509978,
 10.10701483539308,
 9.20243